In [ ]:
# Admin stuff, please ignore
from IPython.core.display import HTML
HTML("""
<style>
a code{
    background-color:#296eaa !important;
    color: #eff0f1 !important;
    font-weight:bold;
}
</style>
""")

# Advanced and Interactive Visualization

This week we'll be taking our data management skills we learned through NumPy and Pandas to start making sense of data. Using two libraries that we will learn about below, Bokeh for graphics and Folium for mapping, we'll start to visualize more complex data and also make the leap to web interfaces.

<hr>
<img src="img/Week4_bokeh.jpg">

## What is [Bokeh](https://docs.bokeh.org/en/latest/index.html)?

Bokeh is an interactive visualization library that targets modern web browsers for presentation. It is good for:

* Interactive visualization in modern browsers
* Standalone HTML documents, or server-backed apps
* Expressive and versatile graphics
* Large, dynamic or streaming data
* Easy usage from python (or Scala, or R, or...)

And most importantly:

## <center><i>NO JAVASCRIPT REQUIRED</i></center>
<code><center>although we will learn javascript 😏</center></code>

Bokeh is an interactive visualization library for modern web browsers. It provides elegant, concise construction of versatile graphics, and affords high-performance interactivity over large or streaming datasets. Bokeh can help anyone who would like to quickly and easily make interactive plots, dashboards, and data applications.

If you're note yet convinced, [DataCamp](https://www.datacamp.com/community/blog/bokeh-cheat-sheet-python) says it best:

<blockquote>Bokeh distinguishes itself from other Python visualization libraries such as Matplotlib or Seaborn in the fact that <b>it is an interactive visualization library that is ideal for anyone who would like to quickly and easily create interactive plots, dashboards, and data applications.</b> 

Bokeh is also known for enabling high-performance visual presentation of large data sets in modern web browsers. 

For data scientists, <b>Bokeh is the ideal tool to build statistical charts quickly and easily; But there are also other advantages, such as the various output options and the fact that you can embed your visualizations in applications.</b> And let's not forget that the wide variety of visualization customization options makes this Python library an indispensable tool for your data science toolbox.</blockquote>
<hr>

## Section 0: What can I *do* with Bokeh

Run the cell below to get started. If you don't have Bokeh installed, run `conda install -c bokeh bokeh`.

In [1]:
# Standard imports 

from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [ ]:
## now we will import some sample data that will be used with the tutorials. 

import bokeh.sampledata
bokeh.sampledata.download()

Bokeh.io is the main library we're working with, and `output_notebook()` tells Jupyter to show figures inline, just below the cells.

Let's see a few demos to get a sense of what Bokeh is all about. The first demo shows different auto manufacturers grouped the number of cylinders in their cars and scaled vertically by their MPG.

In [2]:
# Plot a complex chart with intearctive hover in a few lines of code

# Importing modules
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

# Cleaning sample data
df.cyl = df.cyl.astype(str)
df.yr = df.yr.astype(str)

# Grouping sample data
group = df.groupby(by=['cyl', 'mfr'])
source = ColumnDataSource(group)

# Configuring plot parameters
p = figure(plot_width=800, plot_height=300, title="Mean MPG by # Cylinders and Manufacturer",
           x_range=group, toolbar_location=None, tools="")

# Labeling axes 
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Manufacturer grouped by # Cylinders"
p.xaxis.major_label_orientation = 1.2

# Coloring bars
index_cmap = factor_cmap('cyl_mfr', palette=['#2b83ba', '#abdda4', '#ffffbf', '#fdae61', '#d7191c'], 
                         factors=sorted(df.cyl.unique()), end=1)

# Hover state
p.vbar(x='cyl_mfr', top='mpg_mean', width=1, source=source,
       line_color="white", fill_color=index_cmap, 
       hover_line_color="darkgrey", hover_fill_color=index_cmap)

# Adding tooltip
p.add_tools(HoverTool(tooltips=[("MPG", "@mpg_mean"), ("Cyl, Mfr", "@cyl_mfr")]))

# Displaying the plot
show(p)

### There's a lot in that cell, and it's ok if it doesn't make sense just yet. We'll be breaking it down as we go along. 

### Here's another example, showing a grid format of data:

In [3]:
from math import pi
import pandas as pd

from bokeh.io import show
from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data

data['Year'] = data['Year'].astype(str)
data = data.set_index('Year')
data.drop('Annual', axis=1, inplace=True)
data.columns.name = 'Month'

years = list(data.index)
months = list(data.columns)

# reshape to 1D array or rates with a month and year for each row.
df = pd.DataFrame(data.stack(), columns=['rate']).reset_index()

# this is the colormap from the original NYTimes plot
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=df.rate.min(), high=df.rate.max())

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title="US Unemployment ({0} - {1})".format(years[0], years[-1]),
           x_range=years, y_range=list(reversed(months)),
           x_axis_location="above", plot_width=900, plot_height=400,
           tools=TOOLS, toolbar_location='below',
           tooltips=[('date', '@Month @Year'), ('rate', '@rate%')])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

p.rect(x="Year", y="Month", width=1, height=1,
       source=df,
       fill_color={'field': 'rate', 'transform': mapper},
       line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="5pt",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

show(p)      # show the plot

### Last, here's an example of an interactive map from Bokeh:

In [4]:
from bokeh.io import show
from bokeh.models import LogColorMapper
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.unemployment import data as unemployment

palette.reverse()

counties = {
    code: county for code, county in counties.items() if county["state"] == "tx"
}

county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]

county_names = [county['name'] for county in counties.values()]
county_rates = [unemployment[county_id] for county_id in counties]
color_mapper = LogColorMapper(palette=palette)

data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    rate=county_rates,
)

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="Texas Unemployment, 2009", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    tooltips=[
        ("Name", "@name"), ("Unemployment rate)", "@rate%"), ("(Long, Lat)", "($x, $y)")
    ])
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('x', 'y', source=data,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

show(p)

RuntimeError: bokeh sample data directory does not exist, please execute bokeh.sampledata.download()

## There are many examples available for all types of charts. See Bokeh's [gallery page](https://docs.bokeh.org/en/latest/docs/gallery.html) for more.

<hr>

# Section 1: Basic Plotting

This section of the tutorial covers the [`bokeh.plotting`](http://bokeh.pydata.org/en/latest/docs/user_guide/plotting.html) 
interface. This interface is a "mid-level" interface, and the main idea can be described by the statement:

**Starting from simple default figures (with sensible default tools, grids and axes), add markers and other shapes whose visual attributes are tied to directly data.**

We will see that it is possible to customize and change all of the defaults, but having them means that it is possible to get up and running very quickly.

<hr>

# Imports and Setup

When using the [`bokeh.plotting`](http://bokeh.pydata.org/en/latest/docs/user_guide/plotting.html) interface, there are a few common imports:
* Use the [`figure()`](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure) function to  create new plot objects to work with. 
* Call the functions [`output_file()`](http://bokeh.pydata.org/en/latest/docs/reference/resources_embedding.html#bokeh.io.output_file) or [`output_notebook()`](http://bokeh.pydata.org/en/latest/docs/reference/resources_embedding.html#bokeh.io.output_notebook) (possibly in combination) to tell Bokeh how to display or save output. 
* Execute [`show()`](http://bokeh.pydata.org/en/latest/docs/reference/resources_embedding.html#bokeh.io.show) and  [`save()`](http://bokeh.pydata.org/en/latest/docs/reference/resources_embedding.html#bokeh.io.save) to display or save plots and layouts.

In [ ]:
import numpy as np # we will use this later, so import it now

from bokeh.io import output_notebook, show
from bokeh.plotting import figure

import bokeh #import and download sample data, we'll need this later
bokeh.sampledata.download()

Since we're in Jupyter Notebook, we will call `output_notebook()` so that subsequent calls to `show()` will display inline in the notebook. We did this up above for the demo, but as go ahead and call it again below.

In [ ]:
# call output_notebook() here
output_notebook()

# 2: Scatter Plots

Bokeh can draw many types of visual shapes (called *glyphs*), include lines, bars, patches, hex tiles and more. One of the most common visualization tasks is to draw a scatter plot of data using small *marker* glyphs to represent each point. 

In this section you will see how to use Bokeh's various marker glyphsto create simple scatter plots. 

The basic outline is:
* create a blank figure: `p = figure(...)`
* call a glyph method such as `p.circle` on the figure
* `show` the figure

Execute the cell below to create a small scatter plot with circle glyphs:

In [ ]:
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=15, line_color="navy", fill_color="orange", fill_alpha=0.5)

show(p) # show the results

####################### answer 1
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], radius=1, line_color="navy", fill_color="orange", fill_alpha=0.5)

show(p) # show the results
######################## answer 2
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.square([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=15, line_color="navy", fill_color="orange", fill_alpha=0.5)

show(p) # show the results

The `figure()` function above establishes the canvas with the plotwidth and height, assigning it to variable `p`. Then, the `circle()` method takes two arrays (`[1,2,3,4,5]` and `[6,7,2,4,5]`) for the XY coordinates, and parameters for various styles. You can see the effect of the different options for `line_color`, `fill_alpha`, etc. 
<div class='alert alert-block alert-info'><b>^ Exercise: Try changing some of these values and re-executing the cell to update the plot. All Bokeh scatter markers accept <Code>size</code> (measured in screen space units) as a property. Circles in particular also have <code>radius</code> (measured in "data" space units). 
.</b></div>

<div class='alert alert-block alert-info'><b>^ Exercise: Let's try to scatter square markers instead of circles, using the <code>square()</code> method on figures. Change the method used on <code>p</code> in the second line of code to square instead of circle and see what happens. 
.</b></div>


<hr>

There are many marker types available in Bokeh, you can see details and
example plots for all of them in the reference guide by clicking on entries in the list below:

* [asterisk()](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.asterisk)
* [cricle()](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.circle)
* [cirle_cross()](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.circle_cross)
* [cirle_x()](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.circle_x)
* [cross()](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.cross)
* [diamond](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.diamond)
* [diamond_cross](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.diamond_cross)
* [hex](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.hex)
* [inverted_triangle()](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.inverted_triangle)
* [square()](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.square)
* [square_cross()](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.square_cross)
* [square_x()](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.square_x)
* [triangle()](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.triangle)
* [x()](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.x)

Let's try making a scatter plot. Use the sample data below to make a scatterplot of car weight vs mpg.

<div class='alert alert-block alert-info'><b>Exercise: Use the dataframe <code>df</code> imported below to make a scatterplot of car weight versus miles per gallon (mpg).</b></div>

<div class='alert alert-block alert-info'><b>Color palettes</b>: find more color settings for bokeh here: https://docs.bokeh.org/en/latest/docs/reference/palettes.html 

In [ ]:
# EXERCISE: Make a scatter plot using the "autompg" dataset

from bokeh.sampledata.autompg import autompg as df 

df.head()

In [ ]:
### Your code here
# create a new plot using figure


# add a square renderer with a size, color, alpha, and sizes


# show the results

# 3: Line Plots

Another common visualization task is the drawing of line plots. This can be accomplished in Bokeh by calling the `p.line(...)` glyph method as shown below.

In [ ]:
# create a new plot (with a title) using figure
p = figure(plot_width=400, plot_height=400, title="My Line Plot")

# add a line renderer
p.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=2)

show(p) # show the results

<div class='alert alert-block alert-info'><b>Use the X and Y coordinates below to make a line chart.</b></div>

In [ ]:
# Make a line chart of the fibonacci sequence and its square roots

x = np.array([0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811])
y = np.sqrt(x)

# create a new plot (with a title) using figure


# add a line renderer and assign line_width


# show plot



Like the scatterplot, the `line()` methods takes X and Y coordinates as arrays, then parameters for styling the plot.

In addition to `line_width`, there are other options such as `line_color` or `line_dash` that can be set. 

<div class='alert alert-block alert-info'><b>^ Exercise: Try setting <a href="http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.line" target="_blank">some of the other properties of line</a> and re-running the cell above.</b></div>

### Datetime axes

It's often the case that timeseries data is represented by drawing lines. Let's look at an example using the "glucose" data set, which is available in a Pandas dataframe:

In [ ]:
from bokeh.sampledata.glucose import data
data.head()

We'd like to plot a subset of this data, and have a nice datetime axis as well. We can ask Bokeh for a datetime axis by passing `x_axis_type="datetime"` to the call to `figure`. This is shown below, as well as configuration of a some other options such as plot dimensions, axis titles, and grid line properies. 

In [ ]:
# reduce data size to one week
week = data.loc['2010-10-01':'2010-10-08']

p = figure(x_axis_type="datetime", title="Glocose Range", plot_height=350, plot_width=800)

# modify properties
p.xgrid.grid_line_color=None
p.ygrid.grid_line_alpha=0.5
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Value'

# assign mark
p.line(week.index, week.glucose)

show(p)

In this cell, we introduced a `title` and `x_axis_type` in addition to the `plot_height` and `plot_width` when we assign the figure to `p`. You can see that each line after applies a different property, like modifying the `xgrid` lines and adding axis labels. The method call `p.line(...)` assigns a marker to the plot, and `show(p)` displays it. 

<div class='alert alert-block alert-info'><b>Exercise (below): Look at the AAPL data from bokeh.sampledata.stocks and create a line plot using it with a datetime x axis.</b></div>

In [ ]:
# EXERCISE: Look at the AAPL data from bokeh.sampledata.stocks and create a line plot using it
from bokeh.sampledata.stocks import AAPL

dates = np.array(AAPL['date'], dtype=np.datetime64) # convert date strings to real datetimes

AAPL.keys()

############################# answer


# 4: ColumnDataSource

Instead of giving marker methods like `line()` and `circle()` a slice of a dataframe or array, we can use Bokeh's `ColumnDataSource()` function to manage data. The [`ColumnDataSource()`](https://docs.bokeh.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource) is the core of most Bokeh plots, providing the data that is visualized by the glyphs of the plot. With the [`ColumnDataSource()`](https://docs.bokeh.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource), it is easy to share data between multiple plots and widgets, such as the [`DataTable()`](https://docs.bokeh.org/en/latest/docs/reference/models/widgets.tables.html#bokeh.models.widgets.tables.DataTable). When the same ColumnDataSource is used to drive multiple renderers, selections of the data source are also shared. Thus it is possible to use a select tool to choose data points from one plot and have them automatically highlighted in a second plot (see [`Linked selection`](https://docs.bokeh.org/en/latest/docs/user_guide/data.html#userguide-data-linked-selection)).

At the most basic level, a [`ColumnDataSource()`](https://docs.bokeh.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource) is simply a mapping between column names and lists of data. The [`ColumnDataSource()`](https://docs.bokeh.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource) takes a data parameter which is a dict, with string column names as keys and lists (or arrays) of data values as values. If one positional argument is passed in to the [`ColumnDataSource()`](https://docs.bokeh.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource) initializer, it will be taken as data. Once the [`ColumnDataSource()`](https://docs.bokeh.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource) has been created, it can be passed into the source parameter of plotting methods which allows you to pass a column’s name as a stand in for the data values:

In [ ]:
# import modules
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

# assigning data as a dict
data = {'x_values': [1, 2, 3, 4, 5],
        'y_values': [6, 7, 2, 3, 6]}

# using ColumnDataSource to bridge dictionary into Bokeh
source = ColumnDataSource(data=data)

p = figure(width=200, height=200)

# calling source=source, and keys as string for X and Y values
p.circle(x='x_values', y='y_values', source=source)

show(p)

<div class='alert alert-block alert-info'><b>Exercise (below): Convert your scatterplot of car weight vs MPG to use <code>ColumnDataSource()</code> instead of columns directly from the dataframe.</b></div>

In [ ]:
# Exercise: convert your scatterplot of car weight vs MPG to use ColumnDataSource
############################ answer

# create an empty plot


# create an new dictionary with car weight and car mpg, and put into ColumnDataSource


# add a square renderer with a size, color, alpha, and sizes (try color="firebrick")


# show the results


# 5: Hex Tiling

Bokeh supports drawing low level hex tilings using [axial coordinates](https://www.redblobgames.com/grids/hexagons/#coordinates-axial) and the `hex_tile` method, as described in the [Hex Tiles](https://bokeh.pydata.org/en/latest/docs/user_guide/plotting.html) section of the User's Guide.  However, one of the most common uses of hex tilings is to visualize binning. Bokeh encapsulates this common operation in the `hexbin` function, whose output can be passed directly to `hex_tile` as seen below.

In [ ]:
from bokeh.palettes import Viridis256 # importing the Viridis color pallete
from bokeh.util.hex import hexbin # importing hexbin function

# creating random data
n = 50000
x = np.random.standard_normal(n)
y = np.random.standard_normal(n)

# binning data
bins = hexbin(x, y, 0.1)

# color map the bins by hand, will see how to use linear_cmap later
color = [Viridis256[int(i)] for i in bins.counts/max(bins.counts)*255]

# match_aspect ensures neither dimension is squished, regardless of the plot size
p = figure(tools="wheel_zoom,reset", match_aspect=True, background_fill_color='#440154', width=400, height=400)
p.grid.visible = False

# calling the hex_tile() method
p.hex_tile(bins.q, bins.r, size=0.1, line_color=None, fill_color=color)

show(p)



<div class='alert alert-block alert-info'><b>^ Exercise: Experiment with the size parameter to hexbin, and using different data as input</b></div>

In [ ]:
# Your code here:
# creating random data
n = 50000
x = np.random.standard_normal(n)
y = np.random.standard_normal(n)

# binning data


# color map the bins by hand, will see how to use linear_cmap later


# match_aspect ensures neither dimension is squished, regardless of the plot size


# calling the hex_tile() method


#show plot

# 6: Plots with Multiple Glyphs 

It should be noted that is possible to combine more than one glyph on a single figure. When multiple calls to glyph methods happen on a single figure, the glyphs are draw in the order called, as shown below.

In [ ]:
# set up some data
x = [1, 2, 3, 4, 5]
y = [6, 7, 8, 7, 3]

# create a new plot with figure
p = figure(plot_width=400, plot_height=400)

# add both a line and circles on the same plot
p.line(x, y, line_width=2)
p.circle(x, y, fill_color="white", size=8)

show(p) # show the results

<div class='alert alert-block alert-info'><b> Exercise: Use any of the sample data to make your own chart with multiple glyphs together.</b></div>

In [ ]:
# Exercise: create your own plot with multiple glyphs together!

############################# answer

from bokeh.sampledata.stocks import AAPL
dates = np.array(AAPL['date'], dtype=np.datetime64) # convert date strings to real datetimes
AAPL.keys()

# create an empty figure, assign x_axis_type as "datetime" and title


# add multiple glyphs together


# show plot



# 7: Styling and Theming

This section will cover how to configure various visual aspects of our plots, and how to find out more about what an be configured. 

Before we get started, it's useful to describe how colors and properties are specified in Bokeh. 

## Colors

There are many places where you may need to specify colors. Bokeh can accept colors in a variety of different ways:

* any of the [147 named CSS colors](http://www.w3schools.com/cssref/css_colornames.asp), e.g ``'green'``, ``'indigo'``
* an RGB(A) hex value, e.g., ``'#FF0000'``, ``'#44444444'``
* a 3-tuple of integers *(r,g,b)* between 0 and 255
*  a 4-tuple of *(r,g,b,a)* where *r*, *g*, *b* are integers between 0 and 255 and *a* is a floating point value between 0 and 1



## Properties

Regardless of how a Bokeh plot is created, styling the visual aspects of the plot can always be accomplished by setting attributes on the Bokeh objects that comprise the resulting plot. Visual properties come in three kinds: line, fill, and text properties. For full information with code and examples see the [Styling Visual Properties](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html) section of the user guide. 

----

### Line Properties

Set the visual appearance of lines. The most common are ``line_color``, ``line_alpha``, ``line_width`` and ``line_dash``.

### Fill Properties

Set the visual appearance of filled areas: ``fill_color`` and ``fill_alpha``.

### Text Properties

Set the visual appearance of lines of text. The most common are ``text_font``, ``text_font_size``, ``text_color``, and ``text_alpha``.

Sometimes a prefix is used with property names, e.g. to distinguish between different line properties on the same object, or to give a more meaningful name. For example, to set the line width of the plot outline, you would say ``myplot.outline_line_width = 2``. 

<hr>

To start, here is a simple plot with some new visual styles:

In [ ]:
# assign and call figure()
p = figure(plot_width=400, plot_height=400)

# modify plot outline
p.outline_line_width = 7
p.outline_line_alpha = 0.3
p.outline_line_color = "navy"

# call the marker
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

# show
show(p)

### Selection and non-selection visuals

You can also control how glyphs look when there are selections involved. The set of "selected" points is displayed according to the optional `.selection_glyph` property of a `GlyphRenderer`:
```python
r.selection_glyph = Circle(fill_alpha=1, fill_color="firebrick", line_color=None) 
```
When there is a non-empty selection, the set of "unselected: points is displayed according to the optional  `.nonselection_glyph` property of a `GlyphRenderer`:
```python
r.nonselection_glyph = Circle(fill_alpha=0.2, fill_color="grey", line_color=None) 
```

When using the `bokeh.plotting` interface, it is easier to pass these visual properties to the glyph methods as shown below. The glyph method will create the selection or nonselection glyphs and attach them to the renderer for you. 

In [ ]:
p = figure(plot_width=400, plot_height=400, tools="tap", title="Select a circle")
renderer = p.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=50,

                    # set visual properties for selected glyphs
                    selection_color="firebrick",

                    # set visual properties for non-selected glyphs
                    nonselection_fill_alpha=0.2,
                    nonselection_fill_color="grey",
                    nonselection_line_color="firebrick",
                    nonselection_line_alpha=1.0)

show(p)

## Color Mapping

In addition to placing marks in space and over time, we can use Bokeh to assign colors for values. The example below uses [`LinearColorMapper()`](https://docs.bokeh.org/en/latest/docs/reference/models/mappers.html#bokeh.models.mappers.LinearColorMapper) to assign colors in a range based on the minimum and maximum values of an array. This function takes three arguments: the palette (either [predefined](https://docs.bokeh.org/en/latest/docs/reference/palettes.html) or custom), the low point, and the high point. This establishes the translation between values to colors. You'll want to assign this to a variable to use later. 

Next, you can call [`transform()`](https://docs.bokeh.org/en/latest/docs/reference/transform.html#bokeh.transform.transform) under the color argument of your mark (in this case `circle()`) to transform the data from a number to a color. We're using the weight column of the sample data: `color=transform('weight', color_mapper)`. The `weight` column is `transform`ed from a number to a color based on the range in the `color_mapper` variable.

In [ ]:
# import functions
from bokeh.sampledata.autompg import autompg
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform

# color mapping
source = ColumnDataSource(autompg)
color_mapper = LinearColorMapper(palette="Viridis256", low=autompg.weight.min(), high=autompg.weight.max())

# assign figure() to p
p = figure()
p.circle(x='hp', y='mpg', color=transform('weight', color_mapper), size=20, alpha=0.6, source=autompg)


# show
show(p)

<div class='alert alert-block alert-info'><b>Exercise (below): Below is a scatterplot of flower petal length vs width. Add color to the plot based on the <code>sepal_length</code> column. of the data</b></div>

In [ ]:
from bokeh.sampledata.iris import flowers as df_flower
df_flower.head()

In [ ]:
source = ColumnDataSource(df_flower)
# your code here: assign color_mapper with linearColorMapper
color_mapper

#create plot with circles or squares, use the color_mapper you just created to add color to your plot based on the sepal_length column.
p = figure(plot_width=400, plot_height=400)
p.circle('petal_length', 'petal_width', source=source)
show(p)



It is also possible to specify the visual appearance of glyphs when they are "inspected", e.g. by a hover tool. This is accomplished by setting an optional `hover_glyph` on the glyph renderer:
```python
r.hover_glyph = Circle(fill_alpha=1, fill_color="firebrick", line_color=None) 
```
Or if using `bokeh.plotting` glyph methods, by passing `hover_fill_alpha`, etc. to the glyph method. Lets look at an example that works together with a `HoverTool` configured for "hline" hit-testing.

In [ ]:
from bokeh.models.tools import HoverTool
from bokeh.sampledata.glucose import data

# selecting one day of data
subset = data.loc['2010-10-06']

# x-axis, index as a series
x = subset.index.to_series()

# y-axis, glucose
y = subset['glucose']

# Basic plot setup
p = figure(width=600, height=300, x_axis_type="datetime", title='Hover over points')

# assign the marker -- line # the number following line_dash is the length of dash and space
p.line(x, y, line_dash="1 3", line_width=1, color='gray')

# assign the marker -- circle -- which has the hover states
cr = p.circle(x, y, size=20,
              fill_color="grey", hover_fill_color="firebrick",
              fill_alpha=0.05, hover_alpha=0.3,
              line_color=None, hover_line_color="white")

# adding interactivity
p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='hline'))#change here for exercise

show(p)


Now we're cooking. 

<div class='alert alert-block alert-info'><b>^ Exercise: Try changing some of the parameters above. Change the colors and fill alpha. Change the <code>mode</code> to <code>vline</code> under the <code>add_tools()</code> method and see what happens!</b></div>

# 8: Axes

Next we will take a look at [stlying of Axes](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#axes).

To style axes, you first must get ahold of `Axis` objects. The simplest way is to use some convenience methods on `Plot`: [axis](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.Figure.axis), [xaxis](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.Figure.xaxis), and [yaxis](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.Figure.yaxis). These methods return lists of axis objects:

```
>>> p.xaxis
[<bokeh.models.axes.LinearAxis at 0x106fa2390>]
```
However, you can set properties on all the elements of the list as if it was a single object:
```
p.xaxis.axis_label = "Temperature"
p.axis.major_label_text_color = "orange"
```
These are referred to as "splattable" lists, and tab completion works on them as well. 

<div class='alert alert-block alert-info'><b> Exercise (below): Type <code>p.xaxis.<i>press tab key</i></code></b></div>

In [ ]:
# EXERCISE Try out tab completion.
# Type p.xaxis.<press tab key> to see a list of attributes that can be set.


Now, let's see a few basic properties in action.

In [ ]:
# assign the figure
p = figure(plot_width=400, plot_height=400)

# call the marker
p.asterisk([1,2,3,4,5], [2,5,8,2,7], size=12, color="olive")

# change just some things about the x-axes
p.xaxis.axis_label = "Temp"
p.xaxis.axis_line_width = 3
p.xaxis.axis_line_color = "red"

# change just some things about the y-axes
p.yaxis.axis_label = "Pressure"
p.yaxis.major_label_text_color = "orange"
p.yaxis.major_label_orientation = "vertical"

# change things on all axes
p.axis.minor_tick_in = 1
p.axis.minor_tick_out = 0

show(p)

<div class='alert alert-block alert-info'><b>Exercise: Take one of your previous charts in this notebook and add axis labels.</b></div>

In [ ]:
############################# answer
from bokeh.models.tools import HoverTool
from bokeh.sampledata.glucose import data

# selecting one day of data
subset = data.loc['2010-10-06']

# x-axis, index as a series
x = subset.index.to_series()

# y-axis, glucose
y = subset['glucose']

# Basic plot setup
p = figure(width=600, height=300, x_axis_type="datetime", title='Hover over points')

# assign the marker -- line
p.line(x, y, line_dash="4 4", line_width=1, color='gray')

# assign the marker -- circle -- which has the hover states
cr = p.circle(x, y, size=20,
              fill_color="grey", hover_fill_color="yellow",
              fill_alpha=0.05, hover_alpha=0.3,
              line_color=None, hover_line_color="white")

# adding interactivity
p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='hline'))

# adding axis labels
p.xaxis.axis_label = "Time"
p.xaxis.axis_line_width = 3

# change just some things about the y-axes
p.yaxis.axis_label = "Blood Glucose"
p.axis.axis_line_width = 2

show(p)

# Overview: Annotations

Sometimes we want to add visual cues (boundary lines, shaded regions, labels and arrows, etc.) to our plots to call out some feature or other. Bokeh has several annotation types available for uses like this. Typically to add annotations we create the "low level" annotation object directly, and add it to our plot using `add_layout`. Let's take a look at some specific examples.


# 8: Labels

The `Label` annotation allows you to easily attach single text labels to plots. The position and text to display are configured as `x`, `y`, and `text`:

```python
Label(x=10, y=5, text="Some Label")
```

By default the units are in "data space" but `x_units` and `y_units` maybe set to `"screen"` to position the label relative to the canvas. Labels can also accept `x_offset` and `y_offset` to offset the final position from `x` and `y` by a given screen space distance. 

`Label` objects also have standard text, line (`border_line`) and fill (`background_fill`) properties. The line and fill properties apply to a bounding box around the text:

```python
Label(x=10, y=5, text="Some Label", text_font_size="12pt", 
      border_line_color="red", background_fill_color="blue")
```

In [ ]:
from bokeh.models.annotations import Label
from bokeh.plotting import figure

p = figure(x_range=(0,10), y_range=(0,10))
p.circle([2, 5, 8], [4, 7, 6], color="olive", size=10)

label = Label(x=5, y=7, x_offset=12, text="Second Point", text_baseline="middle")
p.add_layout(label)

show(p)

## LabelSet

The `LabelSet` annotation allows you to create many labels at once, for instance if you want to label an entire set of scatter markers. They are similar to `Label`, but they can also 
accept a `ColumnDataSource` as the `source` property, and then `x` and `y` may refer to columns in the data source, e.g. `x="col2"` (but may also still be fixed values, e.g. `x=10`).

In [ ]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet


source = ColumnDataSource(data=dict(
    temp=[166, 171, 172, 168, 174, 162],
    pressure=[165, 189, 220, 141, 260, 174],
    names=['A', 'B', 'C', 'D', 'E', 'F']))

p = figure(x_range=(160, 175))
p.scatter(x='temp', y='pressure', size=8, source=source)
p.xaxis.axis_label = 'Temperature (C)'
p.yaxis.axis_label = 'Pressure (lbs)'

labels = LabelSet(x='temp', y='pressure', text='names', level='glyph',
                  x_offset=5, y_offset=5, source=source, render_mode='canvas')
                #here the render_mode could change to "css" if you want to specify more fonts, styles:
                #refer to: https://docs.bokeh.org/en/latest/docs/user_guide/annotations.html



p.add_layout(labels)

show(p)

<div class='alert alert-block alert-info'><b>Exercise: Take one of your previous charts and add a labelset to it.</b></div>

In [ ]:
##################################Your answer here, the texts here are only for your guidance, no need to follow strictly.
# assign and call figure()


# modify plot outline


# call the marker (p.circle(), p.square(), etc.)


# create a ColumnDataSource with your label set in one key of the dictionary


# Add labelset


# modify layout by adding your labels


# show the plot


# 9: Legends

When plots have multiple glyphs, it is desirable to include a legend to help users interpret what they see. Bokeh can generate legends easily based on the glyphs that are added. 

### Simple Legends

In the simplest case, you can simply pass a string as the `legend_label` argument to a glyph function:
```python
p.circle(x, y, legend_label="sin(x)")
``` 
In this case Bokeh will automatically create a legend that shows a representation of that glyph, labeled by the test you provided. A full example is below.

In [ ]:
import numpy as np
from bokeh.plotting import output_file, show, figure

# creating x and y values for examples
x = np.linspace(0, 4*np.pi, 100)
y = np.sin(x)

# assigning figure() to p
p = figure()

# mark 1: sin(x)
p.circle(x, y, legend_label="sin(x)")
p.line(x, y)

# mark 2: 2*sin(x)
p.line(x, 2*y, line_dash=[4, 4], line_color="orange", line_width=2)# add legend_label here

# mark 3: 3*sin(x)
p.square(x, 3*y, fill_color=None, line_color="green") # add legend_label here
p.line(x, 3*y, line_color="green")

show(p)

<div class='alert alert-block alert-info'><b>^ Exercise: Add legends for marks 2 and 3 in the above figure.</b></div>

In [ ]:
####################### answer
## copy the code above to here and add legend to each mark
import numpy as np
from bokeh.plotting import output_file, show, figure

# creating x and y values for examples
x = np.linspace(0, 4*np.pi, 100)
y = np.sin(x)

# assigning figure() to p
p = figure()

# mark 1: sin(x)


# mark 2: 2*sin(x)


# mark 3: 3*sin(x)

# show plot


## Removing Marks and Marginalia

Sometimes in your data viz or website, you might prefer a clean, unmarked figure. While often the markings Bokeh includes by default can be useful for exploration, a clean final presentation of the data may be best.

Below, we've removed the `xgrid`, `xaxis`, `ygrid`, and `yaxis` by modifying the `visible` property of each. Additionally, we can hide or restrict the `toolbar`.

For more on visibility, see [Bokeh's docs](https://docs.bokeh.org/en/latest/docs/user_guide/styling.html#visible-property).

In [ ]:
import numpy as np
from bokeh.plotting import output_file, show, figure

# creating x and y values for examples
x = np.linspace(0, 4*np.pi, 100)
y = np.sin(x)

# assigning figure() to p
p = figure()

# mark 1: sin(x)
p.circle(x, y, legend_label="sin(x)")
p.line(x, y)

# mark 2: 2*sin(x)
p.line(x, 2*y, line_dash=[4, 4], line_color="orange", line_width=2, legend_label="sin(x)*2")

# mark 3: 3*sin(x)
p.square(x, 3*y, fill_color=None, line_color="green", legend_label="sin(x)*3")
p.line(x, 3*y, line_color="green")

# remove X axis and grid
p.xgrid.visible = False
p.xaxis.visible = False

# remove Y axis and gris
p.ygrid.visible = False
p.yaxis.visible = False

# remove border
p.outline_line_color = 'White'

# remove logo and toolbar
p.toolbar.logo = None
p.toolbar_location = None

show(p)

## Color bars

Color bars are especially useful if we vary the color of a glyph according to some color mapping. Bokeh color bars are configured with a color mappewr and added to plots with the `add_layout` method:
```python
color_mapper = LinearColorMapper(palette="Viridis256", low=data_low, high=data_high)
color_bar = ColorBar(color_mapper=color_mapper, location=(0,0))
p.add_layout(color_bar, 'right')
```
The example below shows a complete example that also uses the color mapper to transform the glyph color.

In [ ]:
# import functions
from bokeh.sampledata.autompg import autompg
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform

# color mapping
source = ColumnDataSource(autompg)
color_mapper = LinearColorMapper(palette="Viridis256", low=autompg.weight.min(), high=autompg.weight.max())

# assign figure() to p
p = figure(x_axis_label='Horsepower', y_axis_label='MPG', tools='', toolbar_location=None)
p.circle(x='hp', y='mpg', color=transform('weight', color_mapper), size='mpg', alpha=0.6, source=autompg)

# creating color bar
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title='Weight')
p.add_layout(color_bar, 'right')

# show
show(p)

<div class='alert alert-block alert-info'><b>Exercise: Add a color scale and color bar to one of the previous figures.</b></div>

In [ ]:
############################# Some hints here
from bokeh.models.tools import HoverTool
from bokeh.sampledata.glucose import data

# selecting one day of data
subset = data.loc['2010-10-06']

# x-axis, index as a series
x = subset.index.to_series()

# y-axis, glucose
y = subset['glucose']

# Basic plot setup
p = figure(width=600, height=300, x_axis_type="datetime", title='Hover over points')

#create color_mapper


# assign the marker -- line


# assign the marker -- circle -- which has the hover states



# adding interactivity (add_tools)


# adding axis labels and change axis width for xaxis



# change just some things about the y-axes



# add color bar


# show plot


# 10: Embedding and Exporting

So far we have seen how to generate interactive Bokeh output directly inline in Jupyter notbeooks. It also possible to embed interactive Bokeh plots and layouts in other contexts, such as standalone HTML files, or Jinja templates. Additionally, Bokeh can export plots to static (non-interactive) PNG and SVG formats. 

For now, let's return to the first demo.

In [ ]:
# Plot a complex chart with intearctive hover in a few lines of code

# Importing modules
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

# Cleaning sample data
df.cyl = df.cyl.astype(str)
df.yr = df.yr.astype(str)

# Grouping sample data
group = df.groupby(by=['cyl', 'mfr'])
source = ColumnDataSource(group)

# Configuring plot parameters
p = figure(plot_width=800, plot_height=300, title="Mean MPG by # Cylinders and Manufacturer",
           x_range=group, toolbar_location=None, tools="")

# Labeling axes 
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Manufacturer grouped by # Cylinders"
p.xaxis.major_label_orientation = 1.2

# Coloring bars
index_cmap = factor_cmap('cyl_mfr', palette=['#2b83ba', '#abdda4', '#ffffbf', '#fdae61', '#d7191c'], 
                         factors=sorted(df.cyl.unique()), end=1)

# Hover state
p.vbar(x='cyl_mfr', top='mpg_mean', width=1, source=source,
       line_color="white", fill_color=index_cmap, 
       hover_line_color="darkgrey", hover_fill_color=index_cmap)

# Adding tooltip
p.add_tools(HoverTool(tooltips=[("MPG", "@mpg_mean"), ("Cyl, Mfr", "@cyl_mfr")]))

# Displaying the plot
show(p)

Next, importing the `output_file()` function, we can convert this chart directly into an HTML document.

In [ ]:
from bokeh.io import output_file, show
output_file("plot.html")

...and then show it...

In [ ]:
show(p)

<div class='alert alert-block alert-info'><b>Exercise: Export your favorite figure from this notebook!</b></div>

Your new file, `plot.html` is now ready to go online as part of a larger website or on its own!

Next, we'll look at Folium, a powerful webmapping tool based  on Leaflet.

<hr>
End of Bokeh.